In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("discussion.ipynb")

# Discussion 6

### Due Saturday May 14th, 11:59:59PM


# Regex and NLP

In [1]:
import os
import numpy as np
import pandas as pd
import requests
import time
import re

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from discussion import *

## Regular Expressions

### Resources

**Online Simulators**

 - https://pythex.org/

 - https://regex101.com/
 
**Cheat sheets**

 - https://dsc80.com/resources/other/berkeley-regex-reference.pdf

 - https://www.debuggex.com/cheatsheet/regex/python

 - https://www.dataquest.io/wp-content/uploads/2019/03/python-regular-expressions-cheat-sheet.pdf

**Question 1**: Identify duplicate words in a sentence

Given an input sentence as a string, provide a list of words that are duplicated. If there is no duplication return an empty list

**Hint 1:** Checkout capture groups and backreferencing in regex

**Hint 2:** Use the online simulators with the doctests as test cases for faster experimentation and debugging

In [6]:
# Use capture group number to identify duplicates

In [ ]:
grader.check("q1")

**Question 2**: Extract laptop specifications

Given a df with product description - Return df with added columns of `processor` (i3, i5), `generation` (9th Gen, 10th Gen), `storage` (512 GB SSD, 1 TB HDD), `display_in_inch` (15.6 inch, 14 inch). The below image provides details on column names and the exact patterns.

If there is no specific information present, keep a null (`NaN`) value.

**Hint:** You can write regex patterns in `.str.extract()` pandas methods. Note that this method may return multiple columns based on the number of capture groups present.

<img src='imgs/laptop_specs.PNG'>

In [11]:
# Use 'pd_column.str.extract(r'pattern')' to extract the required pattern
df = pd.read_csv('data/laptop_details.csv')

In [12]:
# don't change this cell -- it is needed for the tests to work
out = laptop_details(pd.read_csv('data/laptop_details.csv'))

In [ ]:
grader.check("q2")

## Natural Language Processing - Dealing with Text Data

- Unstructured data is everywhere - Everything you read, see and listen
- Quantifying text data and extracting features from it is important to generate insights and build models


- Text representation ia a huge area of study - Representing a piece of text as a vector of numbers (BoW, TF-IDF, semantic embeddings etc.)

- In this section, we will focus on Bag-of-Words representations using uni-grams and bi-grams

Let's use the musical instuments reviews dataset which contains information on reviews and ratings.

In [18]:
review_df = pd.read_csv('data/musical_instruments_reviews.csv')
review_df

### N-grams in text

 - uni-gram consists of a single word from a text sequence
 - Extending this, an n-gram consists of consecutive 'n' words from a text sequence
 - Eg. For `text = 'i love data science'`, uni-grams are `['i', 'love', 'data', 'science']`, bi-grams are `['i love', 'love data', 'data science']`

### Getting the uni-grams and their counts

In [19]:
# First normalize the reviews by converting to lower and removing all puntuations
reviews = review_df['reviewText'].str.lower().str.replace('[^\w\s]','', regex=True)
reviews = reviews.tolist()
# reviews

In [20]:
# Getting all the unigrams from all the reviews
unigrams = []
for review in reviews:
    words = review.split()
    unigrams.extend(words)
unigrams[:10]

In [21]:
# Getting unigram counts
pd.Series(unigrams).value_counts()

- Does this make sense? Both the values and their counts?
- What are the positives/drawbacks of using unigram bag-of-words for text representations?

### How does 'reviewText' differ from 'summary'?

In [22]:
reviews = review_df['summary'].str.lower().str.replace('[^\w\s]','', regex=True)
reviews = reviews.tolist()

# Getting all the unigrams from all the reviews
unigrams = []
for review in reviews:
    words = review.split()
    unigrams.extend(words)

pd.Series(unigrams).value_counts()

**Question 3**: Create bi-gram counts of the whole reviews text corpus.

Given a DataFrame like `review_df` and a column string (either `reviewText` or `summary`),
return a Series with bi-gram counts of that column sorted in descending order. The index of the series should be a tuple of bi-grams and the value should indicate the count of times that bi-gram appears in the whole corpus.

Perform the text normalization (lower case conversion and removing all punctuations) like we did in the uni-gram case before creating bi-gram counts.

**Hint:** Use splitting and zipping to create bi-gram combinations

In [24]:
review_df.head()

In [25]:
# don't change this cell -- it is needed for the tests to work
out_bigrams_text = bigram_counts(pd.read_csv('data/musical_instruments_reviews.csv'), 'reviewText')
out_bigrams_summary = bigram_counts(pd.read_csv('data/musical_instruments_reviews.csv'), 'summary')

In [ ]:
grader.check("q3")

### Bag-of-Words

- The bag of words model represents texts (e.g. review, summary) as vectors of word counts.
- It is called 'bag of words' because it doesn't consider order.

### Creating the Bag-of-Words Count Matrix

Let's create a BoW count matrix of 'summary' using 'bi-grams'

In [36]:
out_bigrams_summary = bigram_counts(pd.read_csv('data/musical_instruments_reviews.csv'), 'summary')
out_bigrams_summary

In [37]:
reviews = review_df['summary'].str.lower().str.replace('[^\w\s]','', regex=True)
# reviews = reviews.tolist()

# We can reduce sparsity in representations by filtering the bigrams as well.
k = 1000

counts_dict = {}
for bigram in out_bigrams_summary.index[:k]:
    bigram = ' '.join(bigram)
    regex_pattern = fr'\b{bigram}\b'
    counts_dict[bigram] = reviews.str.count(regex_pattern).astype(int).tolist()
    
counts_df = pd.DataFrame(counts_dict)
counts_df

In [38]:
counts_df = pd.concat([reviews.to_frame(), counts_df], axis=1).set_index('summary')
counts_df

### TF-IDF (Term Frequency - Inverse Document Frequency)

- Addresses the BoW drawback of giving high weightage to common words
- TF-IDF tries to **give high weightage to words that are unique to that particular document**
- For comparison, BoW is simply TF


- TF-IDF = Term Frequency * Inverse Document Frequency
    - TF is a function of that document
    - IDF is a function of the corpus
    

- Refer to Lecture 19 for detailed explanations and calculations

### Questions to Ponder

- Pros and cons of Bag-of-Words? Can there be better representations than just counts?
- Pros and cons of TF-IDF?
- Pros and cons of uni-grams vs. bi-grams? Which are suitable in what cases?
- Pros and cons of long text vs. short text?
- What combinations of above might work well?

## Congratulations! You're done!

* Submit your `.py` file to Gradescope. Note that you only need to submit the `.py` file; this notebook should not be uploaded. Make sure that all of your work is in the `.py` file and not here by running the doctests: `python -m doctest discussion.py`.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()